## Imports and definitions

In [1]:
import sys, os, glob, time, requests, json
from time import time, strftime
from tqdm import tqdm, trange
from IPython.display import Audio

import numpy as np
from numpy.random import *
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import folium
from folium import plugins, Map, CircleMarker

In [2]:
dataDir = '../data/'
buildDir = '../build/'
stationsDir = '../data/stations/'

downloadsDir = '../downloads/'
metadataDir = '../metadata/'
archiveDir = '../archive/'

sound = 'sound-3.wav'

maxNormalTime = 6 * 60 * 60

## Analyze data

In [3]:
stations = pd.read_csv(os.path.join(metadataDir, 'stations.csv'), index_col='id')
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 72 to 3249
Data columns (total 5 columns):
latitude       508 non-null float64
longitude      508 non-null float64
stationName    508 non-null object
statusValue    508 non-null object
totalDocks     508 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 23.8+ KB


In [4]:
%%time

dfArray = []

files = glob.glob(os.path.join(downloadsDir, '*.csv'))
for fileName in tqdm(files):
    df = pd.read_csv(fileName)
    dfArray.append(df)

100%|██████████| 12/12 [00:25<00:00,  2.56s/it]


CPU times: user 23 s, sys: 2.15 s, total: 25.2 s
Wall time: 25.2 s


In [5]:
%%time
dfFull = pd.concat(dfArray, ignore_index=True)
dfFull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9937969 entries, 0 to 9937968
Data columns (total 15 columns):
tripduration               int64
starttime                  object
stoptime                   object
start station id           int64
start station name         object
start station latitude     float64
start station longitude    float64
end station id             int64
end station name           object
end station latitude       float64
end station longitude      float64
bikeid                     int64
usertype                   object
birth year                 float64
gender                     int64
dtypes: float64(5), int64(5), object(5)
memory usage: 1.2+ GB
CPU times: user 2.38 s, sys: 1.27 s, total: 3.65 s
Wall time: 3.66 s


In [6]:
reset_selective dfArray

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Pivot table

In [7]:
fieldsToExtract = ['tripduration', 'start station id', 'start station name',
                   'start station latitude', 'start station longitude']

df = dfFull.loc[dfFull.tripduration <= maxNormalTime, fieldsToExtract
                ].groupby('start station id').agg(    
    {'tripduration': {'dep_count': 'count',
                      'dep_mean': 'mean',
                      'dep_min': 'min',
                      'dep_max': 'max',
                      'dep_median': 'median'
                      },
     'start station name': {'name': 'first'},
     'start station latitude': {'latitude': 'first'},
     'start station longitude': {'longitude': 'first'}})

df.index.name = 'id'
df.columns = df.columns.get_level_values(1)
df = df[['name', 'latitude', 'longitude', 'dep_count', 'dep_mean', 'dep_min', 'dep_max', 'dep_median']]

dfDep = df

In [8]:
fieldsToExtract = ['tripduration', 'end station id']

df = dfFull.loc[dfFull.tripduration <= maxNormalTime, fieldsToExtract
                ].groupby('end station id').agg(    
    {'tripduration': {'arr_count': 'count',
                      'arr_mean': 'mean',
                      'arr_min': 'min',
                      'arr_max': 'max',
                      'arr_median': 'median'
                      }})

df.index.name = 'id'
df.columns = df.columns.get_level_values(1)
df = df[['arr_count', 'arr_mean', 'arr_min', 'arr_max', 'arr_median']]

dfArr = df

In [9]:
df = dfDep.join(dfArr, how='left').join(stations[['statusValue', 'totalDocks']], how='left')

In [10]:
reset_selective dfArr

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [11]:
reset_selective dfDep

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [13]:
df.loc[3120, 'name'] = 'Center Blvd & Borden Ave'

df.loc[df.statusValue.isnull(), 'statusValue'] = 'In Service'
df.loc[(df.totalDocks.isnull()) | (df.totalDocks == 0), 'totalDocks'] = df.loc[df.totalDocks != 0, 'totalDocks'].mean()

df.totalDocks = df.totalDocks.astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 72 to 3242
Data columns (total 15 columns):
name           488 non-null object
latitude       488 non-null float64
longitude      488 non-null float64
dep_count      488 non-null int64
dep_mean       488 non-null float64
dep_min        488 non-null int64
dep_max        488 non-null int64
dep_median     488 non-null float64
arr_count      488 non-null int64
arr_mean       488 non-null float64
arr_min        488 non-null int64
arr_max        488 non-null int64
arr_median     488 non-null float64
statusValue    488 non-null object
totalDocks     488 non-null int64
dtypes: float64(6), int64(7), object(2)
memory usage: 61.0+ KB


In [14]:
df.to_csv(os.path.join(dataDir, 'trips-pivot.csv'))

## Daily usage

In [7]:
fieldsToExtract = ['start station id', 'end station id']

In [8]:
start = time()
dtindex = pd.DatetimeIndex(dfFull.starttime)
end = time()

print 'Time elapsed:', (end - start)/60, 'min' 

Audio(url=sound, autoplay=True)

Time elapsed: 2272.20612597


In [5]:
dtindex.to_series().to_csv(os.path.join(archiveDir, 'dt-index.csv'), index=False)

In [68]:
dtindexCut = dtindex[dfFull.tripduration <= maxNormalTime]

df = dfFull.loc[dfFull.tripduration <= maxNormalTime, fieldsToExtract]
df = df.groupby([dtindexCut.year, dtindexCut.month, dtindexCut.day, 'start station id']).agg('count')
df_unstacked = df.unstack(level='start station id', fill_value=0)
df_unstacked.columns = df_unstacked.columns.get_level_values(1)
df_unstacked.head(40)

start station id  72    79    82    83    116   119   120   127   128   137   \
2015 1 1            10    15    10     7    28     3     4    41    24     4   
       2            27    22    11     9    72     3     1    52    51    28   
       3            13    15     7     9    21     0     4    19    13     2   
       4            26    27     9     8    50     5     4    33    16     9   
       5            32    27    16    12    78     5     4    65    76    25   
       6            18    21     7     5    59     4     2    42    38    21   
       7            23    16    14     7    51     8     1    45    44    33   
       8            15    14     9    11    53     3     1    24    30    24   
       9            22    15     7     5    45     4     1    32    41    32   
       10           15    14     6     6    37     2     2    29    42     4   
       11            9    15    11     9    45     3     3    31    36     3   
       12           21    17    10     6    48     4     2    38    44    19   
       13           27    34    12     8    74     4     6    67    74    27   
       14           31    21    18    15    86     6     3    60    61    40   
       15           23    29    24    16    91     5     5    68    68    46   
       16           25    33    19     7    86     5     4    60    78    33   
       17           11    17     7     8    48     1     8    39    46     7   
       18            5     3     3     4    13     0     0     6     2     1   
       19           33    15    13    13    63     2     9    61    64    16   
       20           42    25     7    11   100     8     6    93    86    43   
       21           41    31    13    16    88     7     1    61    89    30   
       22           32    28    11    10   102     7     9    81    86    56   
       23           35    36    16    10    79     5     6    78    88    27   
       24           13     5     3     3    41     0     6    26    26     4   
       25           15    24    13     6    51     4     7    52    48     9   
       26            9     7     2     4    21     5     3    21    26    12   
       27            2     0     2     3     4     1     0     7    13     4   
       28           25     8     6     7    35     1     1    33    27    16   
       29           15    14    10     6    41     5     2    40    21    23   
       30           18    12     7     8    45     5     1    34    41    26   
       31            5     6     7     9    40     3     0    21    34     2   
     2 1            10    14    12     5    39     1     2    32    36     9   
       2             3     5     1     2    10     1     2     5     7     1   
       3            15     7     2     1    30     2     1    28    27     7   
       4            25    14     8     7    54     4     4    32    55    26   
       5            26    12     9    14    66     4     1    50    38    27   
       6            18    12     8     3    51     4     2    34    41    24   
       7            17     7     9     3    38     3     5    28    43     3   
       8            13     8    16    14    47     1     6    30    35    11   
       9            19     5     3     6    31     1     2    24    24     8   

start station id  ...   3231  3232  3233  3234  3235  3236  3237  3238  3241  \
2015 1 1          ...      0     0     0     0     0     0     0     0     0   
       2          ...      0     0     0     0     0     0     0     0     0   
       3          ...      0     0     0     0     0     0     0     0     0   
       4          ...      0     0     0     0     0     0     0     0     0   
       5          ...      0     0     0     0     0     0     0     0     0   
       6          ...      0     0     0     0     0     0     0     0     0   
       7          ...      0     0     0     0     0     0     0     0     0   
       8          ...      0     0     0     0     0     0     0     0     0   

In [69]:
df_unstacked.to_csv(os.path.join(dataDir, 'daily-dep-usage.csv'), index=False)
df_unstacked.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 365 entries, (2015, 1, 1) to (2015, 12, 31)
Columns: 488 entries, 72 to 3242
dtypes: int64(488)
memory usage: 1.4+ MB


In [70]:
df_reset_index = df_unstacked.reset_index()
df_reset_index.head()

start station id,level_0,level_1,level_2,72,79,82,83,116,119,120,...,3231,3232,3233,3234,3235,3236,3237,3238,3241,3242
0,2015,1,1,10,15,10,7,28,3,4,...,0,0,0,0,0,0,0,0,0,0
1,2015,1,2,27,22,11,9,72,3,1,...,0,0,0,0,0,0,0,0,0,0
2,2015,1,3,13,15,7,9,21,0,4,...,0,0,0,0,0,0,0,0,0,0
3,2015,1,4,26,27,9,8,50,5,4,...,0,0,0,0,0,0,0,0,0,0
4,2015,1,5,32,27,16,12,78,5,4,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df_reset_index['date'] = df_reset_index.apply(lambda d: str(d.level_0)+'-'+str(d.level_1)+'-'+str(d.level_2), axis=1)
df_reset_index['date'] = pd.DatetimeIndex(df_reset_index.date)

df_reset_index.drop(['level_0', 'level_1', 'level_2'], axis=1, inplace=True)
df_reset_index.head()

start station id,72,79,82,83,116,119,120,127,128,137,...,3232,3233,3234,3235,3236,3237,3238,3241,3242,date
0,10,15,10,7,28,3,4,41,24,4,...,0,0,0,0,0,0,0,0,0,2015-01-01
1,27,22,11,9,72,3,1,52,51,28,...,0,0,0,0,0,0,0,0,0,2015-01-02
2,13,15,7,9,21,0,4,19,13,2,...,0,0,0,0,0,0,0,0,0,2015-01-03
3,26,27,9,8,50,5,4,33,16,9,...,0,0,0,0,0,0,0,0,0,2015-01-04
4,32,27,16,12,78,5,4,65,76,25,...,0,0,0,0,0,0,0,0,0,2015-01-05


In [72]:
if not os.path.exists(stationsDir):
    os.makedirs(stationsDir)
    
for stationID in df_reset_index.columns[:-1]:
    df_reset_index[['date', stationID]].to_csv(os.path.join(stationsDir, '{}.csv'.format(stationID)), index=False)